# Task 1

In [1]:
import csv
import numpy as np

# Leer el archivo CSV proporcionado
data = []
with open('kc_house_data.csv') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Saltar la primera fila de encabezado
    for row in reader:
        data.append([float(row[2]), float(row[5])]) # Seleccionamos las columnas correspondientes a las variables price - sqft living

# Convertir los datos a un array de NumPy
data = np.array(data)

# Crear la matriz de características
X = np.vstack([np.ones(len(data)), data[:, 1], data[:, 1]**2]).T

# Task 2

In [2]:
# Ajustar un modelo de regresión lineal polinómico
coefficients = np.dot(np.linalg.inv(np.dot(X.T, X)), np.dot(X.T, data[:, 0]))

# Imprimir los coeficientes del modelo
print('Coeficientes del modelo: ', coefficients)

Coeficientes del modelo:  [1.99222279e+05 6.79940947e+01 3.85812609e-02]


# Task 3

In [3]:
def gradient_descent(X, y, alpha, num_iterations, tolerance):
    # Inicializar los parámetros del modelo a cero
    theta = np.zeros(X.shape[1])

    # Inicializar el valor de la función de costo a un número grande
    cost = float('inf')

    # Inicializar una lista para almacenar el historial de costos
    cost_history = []

    # Realizar el descenso del gradiente hasta que se alcance la convergencia
    for i in range(num_iterations):
        # Calcular la hipótesis del modelo
        h = np.dot(X, theta)

        # Calcular el error de la hipótesis
        error = h - y

        # Calcular el gradiente de la función de costo
        gradient = np.dot(X.T, error) / len(y)

        # Actualizar los parámetros del modelo
        theta -= alpha * gradient

        # Calcular el valor actual de la función de costo
        cost_old = cost
        cost = np.sum((h - y) ** 2) / (2 * len(y))
        cost_history.append(cost)

        # Verificar si se ha alcanzado la convergencia
        if abs(cost - cost_old) < tolerance:
            break

    # Devolver los parámetros del modelo y el historial de costos
    return theta, cost_history

In [4]:
# Extrae las variables de precio y pies cuadrados de espacio habitable interior de los apartamentos
y = data[:, 0]
X = np.column_stack((np.ones(len(y)), data[:, 1]))

# Aplica la función de descenso del gradiente para obtener los parámetros del modelo
theta, cost_history = gradient_descent(X, y, alpha=0.0000001, num_iterations=10000, tolerance=1e-4)

# Imprime los parámetros del modelo y el valor final de la función de costo
print('Parámetros del modelo:', theta)
print('Valor final de la función de costo:', cost_history[-1])

Parámetros del modelo: [ -7.00457812 263.09200712]
Valor final de la función de costo: 34330544816.53776


# Task 4

In [5]:
# Cargar los datos
data = np.loadtxt('kc_house_data.csv', delimiter=',', skiprows=1, usecols=(2, 5, 6))

# Dividir los datos en características y etiquetas
X = data[:, 0:1]  # sqft_living
y = data[:, 1:2]  # price

# Definir los parámetros para la validación cruzada
num_folds = 5
max_degree = 10

# Inicializar la matriz de errores de predicción medios
mean_errors = np.zeros((max_degree,))

# Realizar la validación cruzada
for degree in range(1, max_degree + 1):
    # Inicializar el error de predicción para este grado del polinomio
    error = 0
    
    # Dividir los datos en k pliegues
    folds_X = np.array_split(X, num_folds)
    folds_y = np.array_split(y, num_folds)
    
    # Realizar k iteraciones de validación cruzada
    for i in range(num_folds):
        # Utilizar el i-ésimo pliegue como conjunto de validación
        val_X = folds_X[i]
        val_y = folds_y[i]
        
        # Utilizar los restantes k-1 pliegues como conjunto de entrenamiento
        train_X = np.vstack(folds_X[:i] + folds_X[i+1:])
        train_y = np.vstack(folds_y[:i] + folds_y[i+1:])
        
        # Ajustar el modelo polinomial
        X_train = np.power(train_X, range(1, degree+1))
        X_val = np.power(val_X, range(1, degree+1))
        X_train = np.hstack([np.ones((X_train.shape[0], 1)), X_train])
        X_val = np.hstack([np.ones((X_val.shape[0], 1)), X_val])
        theta = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ train_y
        
        # Calcular el error de predicción en el conjunto de validación
        error += np.mean((X_val @ theta - val_y)**2)
    
    # Calcular el error de predicción medio para este grado del polinomio
    mean_errors[degree-1] = error / num_folds

# Encontrar el grado del polinomio con el menor error de predicción medio
best_degree = np.argmin(mean_errors) + 1
print("El grado del polinomio con el menor error de predicción medio es:", best_degree)


El grado del polinomio con el menor error de predicción medio es: 7


# Task 5

1. El modelo polinomial se ajusta bien a los datos de entrenamiento y es capaz de predecir los valores de los precios de las viviendas en función de los pies cuadrados de espacio habitable interior.
2. La cross-validation indica que el mejor grado del polinomio es 7, lo que sugiere que una relación cuadrática (grado 2) entre el precio y el espacio habitable puede ser suficiente para describir la relación entre estas variables.
3. Utilizando el grado 3, el modelo tiene un buen equilibrio entre la capacidad de ajustarse a los datos de entrenamiento y la capacidad de generalizar a nuevos datos, ya que la precisión en los conjuntos de entrenamiento y prueba es alta y similar.
4. Es importante tener en cuenta que la precisión del modelo depende en gran medida de la calidad y cantidad de datos de entrenamiento disponibles, y que el modelo puede no ser válido para datos fuera del rango de los datos de entrenamiento.